In [6]:
## DataProcessing and model generation process
import matplotlib.pyplot as plt
import hdf5storage
import numpy as np
from scipy.signal import butter, lfilter
import os
from keras.utils import np_utils
import tensorflow as tf
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, AveragePooling2D, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from scipy import signal
from datetime import datetime

%matplotlib inline

def Re_referencing(eegData, channelNum, sampleNum):
        after_car = np.zeros((channelNum,sampleNum))
        for i in np.arange(channelNum):
            after_car[i,:] = eegData[i,:] - np.mean(eegData,axis=0)
        return after_car

def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

def Standardization(eegData):
    eegData = eegData.T
    mean_on_data = eegData.mean(axis=0)
    std_on_data = eegData.std(axis=0)

    eegData_standardized = (eegData - mean_on_data) / std_on_data
    eegData_standardized = eegData_standardized.T
    return eegData_standardized
    
def MinMax(eegData):
    eegData = eegData.T
    mins = np.min(eegData, axis = 0)
    maxs = np.max(eegData, axis = 0)

    eegData_normalized = (eegData - mins)/(maxs - mins)
    eegData_normalized = eegData_normalized.T
    
    return eegData_normalized

def Epoching(eegData, stims, samplingFreq, channelNum, epochSampleNum, offset, baseline):
        Time_after = np.add(stims,offset).astype(int)
        Time_base = np.add(stims,baseline).astype(int)
        Num = stims.shape[1]
        Epochs = np.zeros((Num, channelNum, epochSampleNum))
        for j in range(Num):
            Epochs[j, :, :] = eegData[:,Time_after[0][j]:Time_after[0][j] + epochSampleNum]
            
            #Baseline Correction
            for i in range(Epochs.shape[1]):
                Epochs[j,i,:] = Epochs[j,i,:] - np.mean(Epochs[j,i,:])
            
            
        return [Epochs,Num]

def Make_Average_Component(EpochsT, NumT, EpochsN, NumN, channelNum, epochSampleNum, componentNum):
#     EpochsT = Standardization(EpochsT)
#     EpochsN = Standardization(EpochsN)

#     EpochsT = Normalization(EpochsT)
#     EpochsN = Normalization(EpochsN)
    
    NumT_Aver = NumT-componentNum
    NumN_Aver = NumN-componentNum
    
    EpochsT_Aver = np.zeros((NumT_Aver, channelNum, epochSampleNum))
    EpochsN_Aver = np.zeros((NumN_Aver, channelNum, epochSampleNum))
    for i in range(NumT_Aver):
        EpochsT_Aver[i, :, :] = np.mean(EpochsT[i:i+componentNum, :, :], axis=0)
    for j in range(NumN_Aver):
        EpochsN_Aver[j, :, :] = np.mean(EpochsN[j:j+componentNum, :, :], axis=0)
        
    return [EpochsT_Aver, NumT_Aver, EpochsN_Aver, NumN_Aver]

def resampling(Epochs, EpochNum, resampleRate, channelNum):
        resampled_epoch = np.zeros((EpochNum, channelNum, resampleRate))
        for i in range(EpochNum):
            for j in range(channelNum):
                resampled_epoch[i,j,:] = signal.resample(Epochs[i,j,:], resampleRate)
        return resampled_epoch

def GenerateP300Data(filename):
        channelNum = 7
        epochSampleNum = 512
        epochNum = 260
        resampleRate = 100
        target = np.zeros((epochNum,channelNum,resampleRate))
        nontarget = np.zeros((epochNum,channelNum,resampleRate))
        for i in np.arange(1,3):
            if (i==2):
                filename = filename + '_2'
            mat = hdf5storage.loadmat(filename)
            eegData = mat['eegData']
            samplingFreq = mat['samplingFreq'][0,0]
            stimsN = mat['stimsN']
            stimsT = mat['stimsT']
            sampleNum = eegData.shape[1]
            channelIndex = [18, 30, 12, 11, 19, 10, 15]
            
            # vr300 7 channel
            # [P4, Fz, Pz, P3, PO8, PO7, Oz]
            # [19, 31, 13, 12, 20, 11, 16]
            
            eegData = eegData[channelIndex]
            eegData = Standardization(eegData)
            eegData = MinMax(eegData)
            ## Preprocessing process
            eegData = Re_referencing(eegData, channelNum, eegData.shape[1])
            
            #Bandpass Filter
            eegData = butter_bandpass_filter(eegData, 0.1, 30, samplingFreq, 4)
        
            #Epoching
            epochSampleNum = int(np.floor(1.0 * samplingFreq))
            offset = int(np.floor(0.0 * samplingFreq))
            baseline = int(np.floor(1.0 * samplingFreq))
            [EpochsT, NumT] = Epoching(eegData, stimsT, samplingFreq, channelNum, epochSampleNum, offset, baseline)
            [EpochsN, NumN] = Epoching(eegData, stimsN, samplingFreq, channelNum, epochSampleNum, offset, baseline)
            
            EpochsN_New = np.zeros((NumT, channelNum, epochSampleNum))
            NumN = NumT
            for j in range(NumN):
                EpochsN_New[j,:,:] = np.mean(EpochsN[j*5:j*5+5, :, :], axis=0)
            
            #Convert to feature vector
            [EpochsT_Aver, NumT_Aver, EpochsN_Aver, NumN_Aver] = Make_Average_Component(EpochsT, NumT, EpochsN_New, NumN, channelNum, epochSampleNum, 20)
            EpochsT_Aver = resampling(EpochsT_Aver, NumT_Aver, resampleRate, channelNum) 
            EpochsN_Aver = resampling(EpochsN_Aver, NumN_Aver, resampleRate, channelNum)
            
            target[130*(i-1):130*i,:,:] = EpochsT_Aver
            nontarget[130*(i-1):130*i,:,:] = EpochsN_Aver
        
        return [target, nontarget]

def main():
        ctime = datetime.today().strftime("%m%d_%H%M")
        root = 'C:\\Users\\hyuns\\Desktop\\2020-2\\캡스톤\\EEGData\\P300Biosemi55\\S'
        filename = ''
        channelNum = 7
        epochSampleNum = 100
        epochNum = 260
        T_all = np.zeros((epochNum*55,channelNum,epochSampleNum))
        N_all = np.zeros((epochNum*55,channelNum,epochSampleNum))
        
        T_path = 'C:\\Users\\hyuns\\Desktop\\2020-2\\캡스톤\\EEGData\\P300Biosemi55\\AllData\\T_all.out'
        N_path = 'C:\\Users\\hyuns\\Desktop\\2020-2\\캡스톤\\EEGData\\P300Biosemi55\\AllData\\N_all.out'
        
        if not os.path.exists(T_path):
            for i in np.arange(1,56):
                if(i<10):
                    filename = root + '0' + str(i)
                else:
                    filename = root + str(i)
                [T_all[epochNum*(i-1):epochNum*i,:,:],N_all[epochNum*(i-1):epochNum*i,:,:]] = GenerateP300Data(filename)
                print("subject {0} is preprocessed".format(str(i)))
            
            T_all = np.reshape(T_all, (epochNum*55,channelNum*epochSampleNum))
            N_all = np.reshape(N_all, (epochNum*55,channelNum*epochSampleNum))

            np.savetxt(T_path, T_all)
            np.savetxt(N_path, N_all)
        else:
            T_all = np.loadtxt(T_path)
            N_all = np.loadtxt(N_path)
        
        T_all = np.reshape(T_all, (epochNum*55,channelNum,epochSampleNum))
        N_all = np.reshape(N_all, (epochNum*55,channelNum,epochSampleNum))
        
        print(T_all.shape, N_all.shape)
        
        X = np.concatenate((T_all, N_all))
        x = np.reshape(X,((epochNum*2)*55,1,channelNum,epochSampleNum));
        y = np.concatenate((np.ones((epochNum*55,1)).astype(int),np.zeros((epochNum*55,1)).astype(int))).ravel()
        y = np_utils.to_categorical(y, 2);
        
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 777)
        
        ##Generating CNN model
        model = Sequential();
        model.add(Conv2D(epochSampleNum, kernel_size=(1, 20),data_format='channels_first',input_shape=(1, channelNum, epochSampleNum)))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Conv2D(epochSampleNum, (channelNum, 1),data_format='channels_first'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(2))
        model.add(Activation('softmax'))
        
        model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['categorical_accuracy']);
        
        early_stopping = EarlyStopping(patience = 3);
        
        fittedModel = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[early_stopping]);
        
        model.save('C:\\Users\\hyuns\\Desktop\\2020-2\\캡스톤\\Zero\\ZeroModel\\Zero_'+ ctime + '.h5')
        model.save_weights('C:\\Users\\hyuns\\Desktop\\2020-2\\캡스톤\\Zero\\ZeroModel\\weight\\Zero_'+ ctime +'_weight.h5')
            
if __name__ == "__main__":
    main()

subject 1 is preprocessed
subject 2 is preprocessed
subject 3 is preprocessed
subject 4 is preprocessed
subject 5 is preprocessed
subject 6 is preprocessed
subject 7 is preprocessed
subject 8 is preprocessed
subject 9 is preprocessed
subject 10 is preprocessed
subject 11 is preprocessed
subject 12 is preprocessed
subject 13 is preprocessed
subject 14 is preprocessed
subject 15 is preprocessed
subject 16 is preprocessed
subject 17 is preprocessed
subject 18 is preprocessed
subject 19 is preprocessed
subject 20 is preprocessed
subject 21 is preprocessed
subject 22 is preprocessed
subject 23 is preprocessed
subject 24 is preprocessed
subject 25 is preprocessed
subject 26 is preprocessed
subject 27 is preprocessed
subject 28 is preprocessed
subject 29 is preprocessed
subject 30 is preprocessed
subject 31 is preprocessed
subject 32 is preprocessed
subject 33 is preprocessed
subject 34 is preprocessed
subject 35 is preprocessed
subject 36 is preprocessed
subject 37 is preprocessed
subject 38

In [17]:
import numpy as np
from scipy.signal import butter, lfilter, sosfiltfilt
import time
import os, glob
import hdf5storage
import shutil
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import signal
from keras.models import load_model

def Re_referencing(eegData, channelNum, sampleNum):
        after_car = np.zeros((channelNum,sampleNum))
        for i in np.arange(channelNum):
            after_car[i,:] = eegData[i,:] - np.mean(eegData,axis=0)
        return after_car

def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

def Make_Average_Component(Epochs, AverSize):
    Num = int(np.floor(Epochs.shape[0]/AverSize))
    
    Epochs_Aver = np.zeros((Num, Epochs.shape[1], Epochs.shape[2]))
    for i in range(Num):
        Epochs_Aver[i, :, :] = np.mean(Epochs[AverSize*i:AverSize*i+AverSize, :, :], axis=0)
    
    return Epochs_Aver
    
def Epoching(eegData, stims, code, samplingRate, nChannel, epochSampleNum, epochOffset,baseline):
        Time = stims[np.where(stims[:,1] == code),0][0]
        Time = np.floor(np.multiply(Time,samplingRate)).astype(int)
        Time_after = np.add(Time,epochOffset).astype(int)
        Time_base = np.add(Time,baseline).astype(int)
        Num = Time.shape
        Epochs = np.zeros((Num[0], nChannel, epochSampleNum))
        for j in range(Num[0]):
            Epochs[j, :, :] = eegData[:, Time_after[j]:Time_after[j] + epochSampleNum]
            
            # Baseline Correction
            for i in range(nChannel):
                Epochs[j, i, :] = Epochs[j, i, :] - np.mean(Epochs[j, i, :])

        Epochs_Aver = np.mean(Epochs, axis=0)

        return Epochs_Aver

def Standardization(eegData):
    eegData = eegData.T
    mean_on_data = eegData.mean(axis=0)
    std_on_data = eegData.std(axis=0)

    eegData_standardized = (eegData - mean_on_data) / std_on_data
    eegData_standardized = eegData_standardized.T
    return eegData_standardized
    
def MinMax(eegData):
    eegData = eegData.T
    mins = np.min(eegData, axis = 0)
    maxs = np.max(eegData, axis = 0)

    eegData_normalized = (eegData - mins)/(maxs - mins)
    eegData_normalized = eegData_normalized.T
    
    return eegData_norma
lized
    
def Convert_to_FeatureVector(Epochs, buttonNum, featureNum):
    Features = np.zeros((buttonNum, featureNum))
    for i in range(buttonNum):
        Features[i, :] = np.reshape(Epochs[i, :, :], (1, featureNum))
    return Features
    
def resampling(Epochs, resampleRate, channelNum):
        resampled_epoch = np.zeros((channelNum, resampleRate))
        for j in range(channelNum):
            resampled_epoch[j,:] = signal.resample(Epochs[j,:], resampleRate)
            
        return resampled_epoch
    
def main():
        # CNN
        Classifier_path = 'C:\\Users\\hyuns\\Desktop\\2020-2\\캡스톤\\Zero\\ZeroModel\\'
        classifier_list = sorted(glob.glob(Classifier_path + '*.h5'), key=os.path.getmtime, reverse=True)
        weight_list = sorted(glob.glob(Classifier_path + '\\weight\\*.h5'), key=os.path.getmtime, reverse=True)
        print("classifer:", classifier_list[0])
        
#         classifier = 'C:\\Users\\hyuns\\Desktop\\2020-2\\캡스톤\\Zero\\ZeroModel\\Zero_1027_1946.h5'
#         weights = 'C:\\Users\\hyuns\\Desktop\\2020-2\\캡스톤\\Zero\\ZeroModel\\weight\\Zero_1027_1946_weight.h5'
        model = load_model(classifier_list[0])
        model.load_weights(weight_list[0]) 
        
        root_path = 'C:\\Users\\hyuns\\Desktop\\2020-2\\캡스톤\\EEGData\\VR300\\0729\\soram\\'
        resampleRate = 100
        
        mat_path = root_path + 'Online\\'
        current_list = sorted(glob.glob(mat_path + '*.mat'), key=os.path.getmtime, reverse=True)
        score = 0
        wrong_ans = []
        
        for mat_file in current_list:
            print(mat_file)
            ans = mat_file[-5]
            
            mat = hdf5storage.loadmat(mat_file)
            channelNames = mat['channelNames']
            eegData = mat['eegData']
            samplingFreq = mat['samplingFreq']
            samplingFreq = samplingFreq[0,0]
            stims = mat['stims']
            channelNum = channelNames.shape
            channelNum = channelNum[1]
            eegData_ = np.transpose(eegData)
            channelNum = 7
            eegData = eegData_[:channelNum]
            buttonNum = 7
            
#             eegData = eegData / 4
            sampleNum = eegData.shape[1]
            
            eegData = Standardization(eegData)
            eegData = MinMax(eegData)
            #Common Average Reference
#             eegData = Re_referencing(eegData, channelNum, sampleNum)
    
            #Bandpass Filter
            eegData = butter_bandpass_filter(eegData, 0.1, 30, samplingFreq, order=4)

            #Epoching
            epochSampleNum = int(np.floor(1.0 * samplingFreq))
            offset = int(np.floor(0.0 * samplingFreq))
            baseline = int(np.floor(1.0 * samplingFreq))
            
            ####### averaging whole epochs
            Epochs_Aver = np.zeros((buttonNum, channelNum, epochSampleNum))
            Epochs_final = np.zeros((buttonNum, channelNum, resampleRate))
            
            featureNum = channelNum*resampleRate
            
            for i in range(buttonNum):
                Epochs_Aver[i] = Epoching(eegData, stims, (i+1), samplingFreq, channelNum, epochSampleNum, offset, baseline)
                Epochs_final[i] = resampling(Epochs_Aver[i], resampleRate, channelNum)
            
            # CNN
            Epochs_final = np.reshape(Epochs_final, (buttonNum, 1, channelNum, resampleRate))
#             Epochs_final = Epochs_final / 3
            Answers = model.predict(Epochs_final)
            answer = np.argmax(Answers[:, 1]) + 1
            answer = np.argmin(Answers[:, 0]) + 1
            
            print(Answers)
            if int(ans) == int(answer): 
                score = score + 1
            else:
                wrong_ans.append(mat_file[mat_file.rfind('\\')+1:mat_file.rfind('_')])
            print('order: ', ans, 'predict: ', answer)
            
        print('wrong answer', wrong_ans)
        print('score:', score)
if __name__ == "__main__":
    main()

classifer: C:\Users\hyuns\Desktop\2020-2\캡스톤\Zero\ZeroModel\Zero_1103_2119.h5
C:\Users\hyuns\Desktop\2020-2\캡스톤\EEGData\VR300\0729\soram\Online\30_2.mat
[[1.0835791e-21 1.0000000e+00]
 [1.2377711e-11 1.0000000e+00]
 [5.1980691e-33 1.0000000e+00]
 [1.3282939e-34 1.0000000e+00]
 [1.2714205e-02 9.8728579e-01]
 [0.0000000e+00 1.0000000e+00]
 [2.9547244e-02 9.7045273e-01]]
order:  2 predict:  6
C:\Users\hyuns\Desktop\2020-2\캡스톤\EEGData\VR300\0729\soram\Online\29_6.mat
[[9.9731666e-01 2.6833811e-03]
 [1.9359235e-05 9.9998069e-01]
 [6.3779293e-09 1.0000000e+00]
 [1.8353982e-23 1.0000000e+00]
 [9.9991822e-01 8.1809019e-05]
 [1.2906704e-14 1.0000000e+00]
 [7.6067628e-37 1.0000000e+00]]
order:  6 predict:  7
C:\Users\hyuns\Desktop\2020-2\캡스톤\EEGData\VR300\0729\soram\Online\28_4.mat
[[1.2607327e-38 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [7.1173253e-30 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [9.9994421e-01 5.5733773e-05]
 [1.7361668e-26 1.0000000e+00]
 [1.0000000e+00 1.9563877e-13]]

[[0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [6.6966390e-26 1.0000000e+00]
 [3.4045904e-22 1.0000000e+00]
 [5.1030833e-37 1.0000000e+00]]
order:  2 predict:  1
wrong answer ['30', '29', '28', '27', '26', '25', '24', '23', '22', '21', '20', '17', '16', '15', '14', '12', '10', '9', '6', '4', '1']
score: 9
